In [ ]:
import pandas as pd
from kiwipiepy import Kiwi
from collections import Counter

# Kiwi 형태소 분석기 생성
kiwi = Kiwi()

# CSV 파일 경로 설정
input_file = '/data/ephemeral/home/hsk/level2-nlp-datacentric-nlp-01/data.csv'
output_file = 'value_counts.csv'

# 데이터 불러오기
df = pd.read_csv(input_file)

# 형태소 분석 후 명사 추출 함수
def extract_nouns(text):
    nouns = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
            nouns.append(token)
    return nouns

# 각 target 별로 상위 50개 명사 추출
top_nouns = {}
for target_value in range(7):
    # 특정 target에 속하는 텍스트 모음
    texts = df[df['target'] == target_value]['text']
    nouns = []

    # 명사 추출 및 빈도수 계산
    for text in texts:
        nouns.extend(extract_nouns(text))
    
    # 상위 50개의 명사 추출
    noun_counts = Counter(nouns)
    top_nouns[f'target_{target_value}'] = [word for word, _ in noun_counts.most_common(50)]

# 데이터프레임으로 변환
value_counts_df = pd.DataFrame(top_nouns)

# CSV 파일로 저장
value_counts_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"{output_file}에 저장 완료")
